In [1]:
'''
Method developed by Anoop Praturu: https://www.biorxiv.org/content/10.1101/2022.10.12.511940v1
Code from Anoop Praturu on Oct 2021
Move from pystan to cmdstanpy based on suggestions by Milo Julis
Edited by Mingchen Yao on May 26 2023
'''

import numpy as np
import matplotlib.pyplot as plt
import cmdstanpy as stan
import scipy.stats as stats
import pickle
%matplotlib inline

/Users/iuliarusu/miniconda3/envs/HMDS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# # if there is anything wrong with comstan: re-install it. Otherwise don't run this cell
# from cmdstanpy import install_cmdstan
# install_cmdstan(overwrite = True)

In [2]:
path = '/Users/iuliarusu/Documents/Sharpee/HMDS-example/model/'
CM_m = stan.CmdStanModel(stan_file=path+'CM.stan')

12:14:32 - cmdstanpy - INFO - compiling stan file /Users/iuliarusu/Documents/Sharpee/HMDS-example/model/CM.stan to exe file /Users/iuliarusu/Documents/Sharpee/HMDS-example/model/CM
12:14:36 - cmdstanpy - INFO - compiled model executable: /Users/iuliarusu/Documents/Sharpee/HMDS-example/model/CM


In [16]:
#example usage:
#input: number of points, dimension they are embedded in, lorentzian cordinates of embedding
# for lorentzian coordinates you can just pass the ['euc'] coordinates from the dictionary of a previous HMDS fit

fdname = './emb5d.pickle'
with open(fdname, 'rb') as file:
    fit = pickle.load(file)

ex_data = {'N':100, 'D':5, 'coords':fit['euc']}

cm_fit = CM_m.optimize(data=ex_data)

11:53:37 - cmdstanpy - INFO - Chain [1] start processing
11:53:37 - cmdstanpy - INFO - Chain [1] done processing


In [17]:
def get_poin(fit):
    ts = np.sqrt(1.0 + np.sum(np.square(fit['euc']), axis=1))
    return (fit['euc'].T / (ts + 1)).T
#translation of x so origin is translated to v
#thus -v is translated to origin, so put in -v if you want v to be the new origin
def trans_poin(v, x):
    dp = v.dot(x)
    v2 = v.dot(v)
    x2 = x.dot(x)
    
    return ((1.0 + 2.0*dp + x2)*v + (1.0 - v2)*x) / (1.0 + 2.0*dp + x2*v2)

#given center of mass fit of new center, return poincare coords of fit points translated so new center is at origin
def re_center(fit, CM_fit):
    p_coords = get_poin(fit)
    CM_poin = CM_fit['CM']/(1.0 + CM_fit['CM_t'])
    
    return np.asarray([trans_poin(-CM_poin, pt) for pt in p_coords])

In [18]:
# get recentered position in poincare ball
cm_fit_dict = {'CM':cm_fit.CM, 'CM_t':cm_fit.CM_t}
poin_recenter = re_center(fit,cm_fit_dict)